## Local avoidance with timer

In [4]:
from Thymio import Thymio
import numpy as np
from threading import Timer
import time

# import Move_Thymio as mvt

In [2]:
th = Thymio.serial(port="\\.\COM5", refreshing_rate=0.1)

In [7]:
vreal_max = 0.1525
def vThymio2vReal(vThymio):
    """
    Input: vThymio = 0 - 500
    Output: vReal = 0 - 0.14 m/s
    """
    
    if vThymio > 500:
        print("Warning: vThymio above 500, saturating vReal to 0.14")
        vReal = vreal_max

    elif vThymio < 0:
        print("Warning: vThymio negative")
        vThymio = 0
    
    else:
        vReal = vThymio*vreal_max/500
    
    return vReal 

def vReal2vThymio(vReal):
    """
    Input: vReal = 0 - 0.14 m/s
    Output: vThymio = 0 - 500
    """
    if vReal > vreal_max:
        print("Warning: vReal above 0.14 m/s, saturating vThymio to 500")
        vThymio = 500

    elif vReal < 0:
        print("Warning: vReal negative")
        vThymio = 0
        
    else:
        vThymio = vReal*500/vreal_max
       
        
    return vThymio

def stop():
    
    """Shutdown all the Thymio motors"""
    
    th.set_var("motor.left.target", 0)
    th.set_var("motor.right.target", 0)
    
def Thymio_rotate(angle, vThymio = 150):

    vReal = vThymio2vReal(vThymio)
    cal_R = 10
    cal_L = 0

    dist_wheel = 95*1e-3 # dist between wheels [m]
    dt = abs(angle)*vReal/(dist_wheel/2)

    if angle > 0:
        th.set_var("motor.left.target", 2**16-(vThymio + cal_L))
        th.set_var("motor.right.target", vThymio + cal_R)
    elif angle < 0:
        th.set_var("motor.left.target", vThymio + cal_L)
        th.set_var("motor.right.target", 2**16-(vThymio + cal_R))
    elif angle == 0:
        stop()
        disp("Warning: angle = 0")


    time.sleep(dt)
    stop()
    
    return dt

def Thymio_translate(l_speed = 150, r_speed = 150):
    """
    Input: speed Thymio = 150 (default)
    Output: Thymio moves (default = forward) at speed indicated (default = 150)  
    """
    r_wheel = 43/2*1e-3 #radius of a wheel [m]
    
    """Calibration part"""
    cal_R = 6 # calibration term for right motor
    cal_L = 0 # calibration term for left motor
    
    if l_speed < 0: l_speed = 2**16 - (l_speed+cal_L)
    elif l_speed > 0: l_speed = l_speed + cal_L
            
    if r_speed < 0: r_speed = 2**16 - (l_speed+cal_R)
    elif r_speed > 0: r_speed = r_speed + cal_R
            
    th.set_var("motor.left.target", l_speed)
    th.set_var("motor.right.target", r_speed)

In [15]:
def local_avoidance(prox_sensors):
    
    """
    Implement Braitenberg strategy
    
    Input: proximity sensor values
    Output: Thymio dodges obstacle by quickly rotating wheel near obstacle
    """
    
    W=np.matrix([[2, 2, -1, -2, -2], 
                    [-2, -2, -1, 2, 2]]) # wight matrix for each front sensor
    
    y = W.dot(prox_sensors)
    y = np.divide(y,10)
    print("y = ",y)

    l_speed = int(np.round(y[0,0]))
    r_speed = int(np.round(y[0,1]))
    
    speed = np.array([l_speed, r_speed])
    
    speed[speed > 500] = 500 # Saturation condition to avoid "bytes must be in range(0, 256)" error
    speed[speed < -500] = -500
    
    print("speed = ", speed)
    
    
    if (l_speed < 0): l_speed = 2**16-abs(l_speed) #convert negative values for Thymio.py
    if (r_speed < 0): r_speed = 2**16-abs(r_speed)

    th.set_var("motor.left.target", l_speed)
    th.set_var("motor.right.target", r_speed)

#     t = time.time()

#     stop()

In [8]:
def FSM_avoidance(verbose=True): #Function to be repeated with timer
    """
    Checks at regular intervals the prox sensor and enters local avoidance if obstacle detected
    """
    prox = False
    tic = time.time()
    t =  time.time()
    operating_time = 10 #sec

    while t-tic <= operating_time:
    
        prox_sensors = th["prox.horizontal"][0:5]
        

        if max(prox_sensors)!=0 and prox == False: #obstacle detected while still in global navigation mode
            
            if verbose: print("Obstacle go local")
            prox = True
        
        elif max(prox_sensors) ==0 and prox == True: #no obstacle in front in local avoidance --> go back to global
            
            if verbose: print("No obstalce go global")
            prox = False

        
        if prox:
            
            local_avoidance(prox_sensors)
        
        elif not prox:
            
            Thymio_translate()
        
        t = time.time()
    
    stop()   

In [10]:
stop()

In [16]:
FSM_avoidance()

Obstacle go local
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -260.]]
speed =  [ 260 -260]
y =  [[ 260. -26

y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
y =  [[-386.9  151.1]]
speed =  [-387  151]
No obstalce go global
Obstacle go local
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -1067.5]]
speed =  [ 500 -500]
y =  [[  565.3 -10

speed =  [-500  104]
y =  [[-930.5  103.9]]
speed =  [-500  104]
y =  [[-930.5  103.9]]
speed =  [-500  104]
y =  [[-930.5  103.9]]
speed =  [-500  104]
y =  [[-930.5  103.9]]
speed =  [-500  104]
y =  [[-930.5  103.9]]
speed =  [-500  104]
y =  [[-930.5  103.9]]
speed =  [-500  104]
y =  [[-930.5  103.9]]
speed =  [-500  104]
y =  [[-930.5  103.9]]
speed =  [-500  104]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804.3  492.1]]
speed =  [-500  492]
y =  [[-804

y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
y =  [[ 207.8 -563. ]]
speed =  [ 208 -500]
No obstalce go global
Obstacle go local
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-421 -217]
y =  [[-420.6 -217.4]]
speed =  [-42

speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.2 -2067. ]]
speed =  [ 500 -500]
y =  [[ 1384.